In [9]:
import torch

# 加载.pt文件
pt_file = "output/original_contrast2/runs-1-703.pt"  # 请替换成你的文件路径
model = torch.load(pt_file, map_location=torch.device('cpu'))  # 读取文件到CPU

# 如果是 state_dict，直接打印各层的名称和维度
if isinstance(model, dict):
    state_dict = model.get('state_dict', model)  # 有些模型可能嵌套在 'state_dict' 字典内
    print("Layers and their shapes:")
    for name, param in state_dict.items():
        print(f"{name}: {param.shape}")
else:
    print("Loaded object is not a state_dict. It might be a complete model.")



Layers and their shapes:
joint_person_embedding: torch.Size([96, 48])
stem.0.weight: torch.Size([4, 2, 1, 1])
stem.0.bias: torch.Size([4])
stem.2.weight: torch.Size([6, 4, 1, 1])
stem.2.bias: torch.Size([6])
stem.4.weight: torch.Size([96, 6, 1, 1])
stem.4.bias: torch.Size([96])
stages.0.blocks.0.transformer.gconv: torch.Size([8, 48, 48])
stages.0.blocks.0.transformer.norm_1.weight: torch.Size([96])
stages.0.blocks.0.transformer.norm_1.bias: torch.Size([96])
stages.0.blocks.0.transformer.mapping.weight: torch.Size([192, 96])
stages.0.blocks.0.transformer.mapping.bias: torch.Size([192])
stages.0.blocks.0.transformer.tconv.weight: torch.Size([24, 3, 7, 1])
stages.0.blocks.0.transformer.tconv.bias: torch.Size([24])
stages.0.blocks.0.transformer.attention.0.relative_position_bias_table: torch.Size([15, 4])
stages.0.blocks.0.transformer.attention.0.relative_position_index: torch.Size([8, 8])
stages.0.blocks.0.transformer.attention.1.relative_position_bias_table: torch.Size([15, 24, 24, 4])
s

In [11]:
import numpy as np
import json
import random
import math
import pickle
from tqdm import tqdm
import os
from concurrent.futures import ThreadPoolExecutor, as_completed
from torch.utils.data import Dataset
from sklearn.metrics import precision_recall_fscore_support


RH = 94
LH = 115
FK = 26
# 定义骨骼连接关系，列表中的元组表示骨骼的连接点
bone = [
    # Head
    (0, 1), (0, 2), (1, 3), (2, 4),
    # Body
    (0, 18),
    (5, 18), (6, 18), (5, 7), (7, 9), (6, 8), (8, 10),
    (17, 18), (18, 19), (19, 11), (19, 12),
    (11, 13), (12, 14), (13, 15), (14, 16),
    # Foot
    (20, 24), (21, 25), (23, 25), (22, 24), (15, 24), (16, 25),
    # Right Hand
    (9, RH+0), (RH+0, RH+4), (RH+0, RH+8), (RH+0, RH+8), (RH+0, RH+12), (RH+0, RH+16), (RH+0, RH+20),
    # Left Hand
    (10, LH + 0), (LH + 0, LH + 4), (LH + 0, LH + 8), (LH + 0, LH + 12), (LH + 0, LH + 16), (LH + 0, LH + 20),
    # Face
    (FK + 33, 0), (FK + 51, FK + 57), (FK + 33, FK + 51), (FK + 28, FK + 33), (FK + 28, FK + 45), (FK + 28, FK + 36)
]

right_hand = np.array([RH+0, RH+4, RH+8, RH+12, RH+16, RH+20])
left_hand = np.array([LH+0, LH+4, LH+8, LH+12, LH+16, LH+20])

right_arm = np.array([5,5,7,7,9,9])
left_arm = np.array([6,6,8,8,10,10])

right_leg = np.array([11,13,15,24,22,20])
left_leg = np.array([12,14,16,25,23,21])

face = np.array([FK+57, FK+51, FK+33, FK+28, FK+45, FK+36])
torso = np.array([18,0,3,4,17,19])

new_idx = np.concatenate(
    (right_hand, left_hand, right_arm, left_arm, right_leg, left_leg, face, torso), axis=-1)


In [15]:
dic = dict()
for i, n in enumerate(list(new_idx)):
    dic[n] = i

In [17]:
dic

{94: 0,
 98: 1,
 102: 2,
 106: 3,
 110: 4,
 114: 5,
 115: 6,
 119: 7,
 123: 8,
 127: 9,
 131: 10,
 135: 11,
 5: 13,
 7: 15,
 9: 17,
 6: 19,
 8: 21,
 10: 23,
 11: 24,
 13: 25,
 15: 26,
 24: 27,
 22: 28,
 20: 29,
 12: 30,
 14: 31,
 16: 32,
 25: 33,
 23: 34,
 21: 35,
 83: 36,
 77: 37,
 59: 38,
 54: 39,
 71: 40,
 62: 41,
 18: 42,
 0: 43,
 3: 44,
 4: 45,
 17: 46,
 19: 47}

In [26]:
O = []
for i in bone:
    print(i)
    if i[0] not in dic.keys() or i[1] not in dic.keys():
        continue
    O.append((dic[i[0]], dic[i[1]]))

(0, 1)
(0, 2)
(1, 3)
(2, 4)
(0, 18)
(5, 18)
(6, 18)
(5, 7)
(7, 9)
(6, 8)
(8, 10)
(17, 18)
(18, 19)
(19, 11)
(19, 12)
(11, 13)
(12, 14)
(13, 15)
(14, 16)
(20, 24)
(21, 25)
(23, 25)
(22, 24)
(15, 24)
(16, 25)
(9, 94)
(94, 98)
(94, 102)
(94, 102)
(94, 106)
(94, 110)
(94, 114)
(10, 115)
(115, 119)
(115, 123)
(115, 127)
(115, 131)
(115, 135)
(59, 0)
(77, 83)
(59, 77)
(54, 59)
(54, 71)
(54, 62)


In [29]:
O

[(43, 42),
 (13, 42),
 (19, 42),
 (13, 15),
 (15, 17),
 (19, 21),
 (21, 23),
 (46, 42),
 (42, 47),
 (47, 24),
 (47, 30),
 (24, 25),
 (30, 31),
 (25, 26),
 (31, 32),
 (29, 27),
 (35, 33),
 (34, 33),
 (28, 27),
 (26, 27),
 (32, 33),
 (17, 0),
 (0, 1),
 (0, 2),
 (0, 2),
 (0, 3),
 (0, 4),
 (0, 5),
 (23, 6),
 (6, 7),
 (6, 8),
 (6, 9),
 (6, 10),
 (6, 11),
 (38, 43),
 (37, 36),
 (38, 37),
 (39, 38),
 (39, 40),
 (39, 41)]